In [1]:
from dispatch_jobs import DB, KEY_PREFIX
import pickle
import logging
import numpy as np

logging.info(f'key prefix is {KEY_PREFIX}')
keys = DB.keys(KEY_PREFIX + '*')

INFO:root:Database connection successful
INFO:root:key prefix is svmph_


In [2]:
# # Changing the path of the pdbbind dataset from hpc to local in the redis database
# # Only needs to be run once

# from tqdm import tqdm

# # original = '/hpc/group/donald/yl708/persistence-diagrams'
# # new = '/home/longyuxi/Documents/mount/persistence-diagrams'

# original = '/hpc/group/donald/yl708/pdbbind/refined-set'
# new = '/home/longyuxi/Documents/mount/pdbbind-dataset/refined-set'

# for key in keys:
#     info = DB.hgetall(key)

#     for k, v in info.items():
#         if original in v:
#             info[k] = v.replace(original, new)

#     DB.hset(key, mapping=info)

In [3]:
# For testing
# keys = keys[:100]

In [4]:
# from gtda.diagrams import PersistenceImage
# from tqdm import tqdm

# pim = PersistenceImage()

# observations = []
# binding_affinities = []

# for key in tqdm(keys[:1000]):
#     info = DB.hgetall(key)
#     if info['finished'] == 'True' and info['error'] == 'False':
#         save_file = info['save_file']

#         with open(save_file, 'rb') as f:
#             d = pickle.load(f)

#         pw_opposition_diagrams = d['pw_opposition_diagrams']  # list of 36 diagrams, each of shape (1, n, 3)
#         other_persistence_diagrams = d['other_persistence_diagrams']  # other_persistence_diagrams: list of 4 diagrams, each of shape (1, n, 3)

#         all_diagrams = pw_opposition_diagrams + other_persistence_diagrams
#         all_images = list(map(lambda x: pim.fit_transform(x), all_diagrams))

#         observations.append(np.array(all_images))

#         binding_affinities.append(float(info['-logKd/Ki']))


# observations = np.array(observations)
# binding_affinities = np.array(binding_affinities)

# with open('observations.npy', 'wb') as f:
#     np.save(f, observations)

# with open('binding_affinities.npy', 'wb') as f:
#     np.save(f, binding_affinities)

with open('observations.npy', 'rb') as f:
    observations = np.load(f)
    print(observations.shape)

with open('binding_affinities.npy', 'rb') as f:
    binding_affinities = np.load(f)
    print(binding_affinities.shape)


100%|██████████| 1000/1000 [02:05<00:00,  7.95it/s]


In [5]:
from sklearn.model_selection import train_test_split

observations = observations.reshape(observations.shape[0], -1)  # Flatten out for SVM

# For testing
observations = observations[:1000]
binding_affinities = binding_affinities[:1000]

X_train, X_test, y_train, y_test = train_test_split(observations, binding_affinities, test_size=0.2, random_state=42)

In [6]:
print(X_train.shape)
print(y_train.shape)

(729, 1200000)
(729,)


In [7]:
from sklearn import svm

# regr = svm.LinearSVR(max_iter=1000, verbose=1)
regr = svm.SVR(verbose=True)

regr.fit(X_train, y_train)


[LibSVM]*
optimization finished, #iter = 468
obj = -965.846581, rho = -5.924855
nSV = 695, nBSV = 657


SVR(verbose=True)

In [8]:
import plotly.express as px
from sklearn.metrics import r2_score

y_hat = regr.predict(X_test)

fig = px.scatter(x=y_test, y=y_hat, labels={'x': 'True binding affinity', 'y': 'Predicted binding affinity'}, title=f'True vs Predicted binding affinity\nR^2 = {r2_score(y_test, y_hat):.2f}, n = {len(y_test)}, RMSE: {np.sqrt(np.mean((y_test - y_hat)**2)):.2f}, Pearson: {np.corrcoef(y_test, y_hat)[0, 1]:.2f}')
fig.show()

In [11]:

fig = px.scatter(x=y_test, y=y_hat, labels={'x': 'True binding affinity', 'y': 'Predicted binding affinity'}, title=f'True vs Predicted binding affinity\nR^2 = {r2_score(y_test, y_hat):.2f}, n = {len(y_test)}, RMSE: {np.sqrt(np.mean((y_test - y_hat)**2)):.2f}, Pearson: {np.corrcoef(y_test, y_hat)[0, 1]:.2f}')
fig.show()

In [9]:
print(f'RMSE: {np.sqrt(np.mean((y_test - y_hat) ** 2)): .4f}')
print(f'Pearson correlation: {np.corrcoef(y_test, y_hat)[0, 1]: .4f}')

RMSE:  1.7507
Pearson correlation:  0.4300


In [10]:
raise Exception

Exception: 

In [ ]:
from gtda.plotting import plot_diagram

diagram = all_diagrams[36]
plot_diagram(diagram[0])

In [ ]:

xt = pim.fit_transform(diagram)



# pim.plot(all_images[10], homology_dimension_idx=1)

# Plot each image in all_images

import os
if not os.path.exists('images'):
    os.mkdir('images')

for o_i in range(10):
    all_images = observations[o_i]
    for i in range(40):
        fig = pim.plot(all_images[i], homology_dimension_idx=1)
        fig.write_image(f'images/{i}.png')

In [ ]:
# The only interesting ones seem to be in images 36-39
# Make SVM out of them

interesting_images = all_images[36:40]



